# 数据标准化与变换数据

# 数据标准化

## 什么是数据标准化？为什么要进行数据标准化？

数据标准化就是通过数学方法，把不同量纲或数量级的数据转化到统一的尺度，使它们更具可比性。

为什么需要数据标准化
- 消除量纲差异：不同指标可能有不同的单位（如米、千克、秒），直接比较会失真。
- 避免数量级影响：某些变量数值范围很大，会在分析中占主导地位，而小范围变量可能被忽视。
- 提高模型效果：在机器学习中，标准化能帮助算法更快收敛，提高准确性。




应用场景
- 机器学习：如 KNN、SVM、神经网络等算法对数据尺度敏感。
- 综合评价：在医疗、经济、教育等领域，多个指标需要统一比较。
- 图像处理：像素值标准化便于模型识别。


## 数据标准化的常用方法

### 离差标准化

离差标准化（也叫 极差标准化 / Min-Max Normalization）是通过数据的 最大值 和 最小值 来进行线性变换，把数据映射到一个固定区间（通常是 [0,1] 或 [-1,1]）。

公式如下：

$$
X'=\frac{X-X_{min}}{X_{max}-X_{min}}
$$ 

- $ X $：原始数据
- $ X_{min} $：样本中的最小值
- $ X_{max} $：样本中的最大值
- $ X' $：标准化后的数据

特点
- 范围固定：通常压缩到 [0,1] 区间，便于比较。
- 保持比例关系：数据之间的相对差异不变。
- 对异常值敏感：如果存在极端值，整个缩放区间会被拉伸。



In [ ]:
import pandas as pd
pay = pd.read_csv('pd_data/user_pay_info.csv', index_col=0)

In [ ]:
# 自定义离差标准化函数
def min_max_scale(data):
    data = (data - data.min()) / (data.max() - data.min())
    return data

In [ ]:
pay_min_max = min_max_scale(pay['每月支出'])

In [ ]:
pay['每月支出']

In [ ]:
pay_min_max

### 标准差标准化数据

标准差标准化（也叫 Z-score 标准化 或 零均值标准化）

是通过数据的 均值 和 标准差 来调整数据，使其转化为一个 均值为 0、标准差为 1 的分布。

公式如下：


$$
 X'=\frac{X-\mu }{\sigma } 
$$

- $ X$：原始数据
- $ \mu $：样本均值
- $ \sigma $ ：样本标准差
- $ X' $：标准化后的数据

特点
- 均值为 0：标准化后的数据中心在零点。
- 方差为 1：数据分布的尺度统一。
- 消除量纲影响：不同单位（如米、千克）的数据可以放在同一模型中比较。
- 对异常值敏感：因为标准差受极端值影响，离差标准化在存在离群点时可能不稳定。




In [ ]:
# 自定义标准差标准化函数
def standard_scaler(data):
    data = (data - data.mean()) / data.std()
    return data

In [ ]:
pay_standard = standard_scaler(pay['每月支出'])

In [ ]:
pay['每月支出']

In [ ]:
pay_standard

### 小数定标标准化

小数定标标准化是通过 移动数据的小数点位置 来实现标准化的一种方法。它的核心思想是：
**让数据的绝对值不超过 1。**

公式如下：
$$
X'=\frac{X}{10^j}
$$

其中：
- $ X $：原始数据
- $ j $：需要移动的小数点位数
- $ X $'：标准化后的数据

选择$ j $的原则是：
$$ \max (|X'|)<1 $$
即标准化后所有数据的绝对值都小于 1。

特点
- 简单直观：只需确定小数点移动的位数。
- 快速计算：不依赖均值、方差或最大最小值。
- 适用范围有限：主要用于数据范围已知且分布较均匀的情况。
- 对异常值敏感：极端值会影响小数点移动的位数。


In [ ]:
# 自定义小数定标标准化函数
import numpy as np
def decimal_scaler(data):
    data = data / 10 ** np.ceil(np.log10(data.abs().max()))
    return data

In [ ]:
pay_decimal = decimal_scaler(pay['每月支出'])

In [ ]:
pay['每月支出']

In [ ]:
pay_decimal

# 变换数据

在数据清洗工作中，数据变换是一个非常重要的环节，它的目的不仅仅是“让数据看起来整齐”，而是为了让后续分析和建模更加科学、准确。

为什么要进行数据变换
- 消除量纲差异
    - 不同字段可能有不同的单位或数量级（如收入以万元计、年龄以岁计），直接比较会失真。通过标准化或归一化，可以让它们处于同一尺度。
- 提高模型性能
    - 许多机器学习算法（如 KNN、SVM、神经网络）对数据的尺度敏感。如果不变换，某些特征可能因为数值范围大而主导模型，导致偏差。
- 满足统计假设
    - 一些统计方法要求数据满足特定分布（如正态分布）。通过对数变换、平方根变换等，可以让数据更接近这些假设条件。
- 减少偏态和异常值影响
    - 数据常常存在偏态分布或极端值。适当的变换（如对数变换）可以压缩大值的影响，使分布更平滑。
- 便于解释和比较
    - 统一后的数据更容易进行跨指标比较，也更容易在可视化中展示。


## 哑变量处理类别类型数据

什么是哑变量处理
- 类别型数据（如性别、颜色、地区）不能直接用于大多数统计模型或机器学习算法，因为它们是非数值型。
- 哑变量处理就是把类别型数据转化为数值型数据的一种方法。
- 核心思想：用 0/1（二进制）变量 来表示某个类别是否出现。

在pandas中，可以使用get_dummies函数对类别类型进行哑变量处理
```
pd.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, drop_first=False)
```

常用参数说明
- data：输入的 DataFrame 或 Series。
- columns：指定要转换的列，默认对所有类别型列进行处理。
- prefix：为生成的哑变量列添加前缀。
- prefix_sep：前缀和类别名之间的分隔符，默认是 _。
- dummy_na：是否为缺失值生成一列，默认 False。
- drop_first：是否删除第一个类别，避免虚拟变量陷阱（多重共线性）。


In [ ]:
import pandas as pd
all_info = pd.read_csv('pd_data/user_all_info.csv')

In [ ]:
live_type = all_info['居住类型']

In [ ]:
live_type

In [ ]:
pd.get_dummies(live_type)

## 离散化连续型变量

在数据分析中，离散化连续型变量（也叫分箱 binning）是一种常见的预处理方法。它的核心思想是：把连续的数值变量划分为若干区间，再用类别型变量来表示。

为什么要离散化连续型变量
- 简化模型
    - 连续变量可能取值范围很大，离散化后可以减少复杂度，让模型更容易处理。
- 增强可解释性
    - 离散化后的变量更容易被人理解。例如，把“年龄”分为“青年、中年、老年”，比直接用数值更直观。
- 处理非线性关系
    - 某些变量与目标变量的关系不是线性的。通过分箱，可以更好地捕捉这种关系。
- 减少噪声影响
    - 连续变量可能受测量误差或波动影响，离散化后能降低噪声对模型的干扰。
- 满足算法需求
    - 一些算法或统计方法（如决策树、朴素贝叶斯）对类别型变量更友好，离散化能提高适用性。


常见离散化方法
- 等宽分箱（等宽法）：把数值范围平均分成若干区间。
- 等频分箱（等频法）：保证每个区间内样本数量大致相同。
- 基于聚类的分箱（聚类分析法）：用聚类算法划分区间。
- 基于业务规则的分箱：如年龄分为“青年、中年、老年”。


### 等宽法

等宽分箱（Equal-width Binning） 是一种常见的连续型变量离散化方法。它的核心思想是：把数据的取值范围按照固定的宽度划分为若干个区间，每个区间的宽度相同。

原理：
假设数据的最小值是 X_{min}，最大值是 X_{max}，需要分成 k 个区间。
每个区间的宽度为：
$$
w=\frac{X_{max}-X_{min}}{k}
$$
然后按照这个宽度，把整个区间划分为：
$$
[X_{min},X_{min}+w),[X_{min}+w,X_{min}+2w),\dots ,[X_{min}+(k-1)w,X_{max}]
$$
特点
- 简单直观：只需确定分箱数量 k，计算区间宽度即可。
- 区间宽度相同：保证划分均匀。
- 样本分布不均衡：如果数据集中在某些区间，可能导致某些箱子样本很多，另一些箱子样本很少。
- 对异常值敏感：极端值会拉大区间范围，导致分箱效果不理想。


在pandas中，提供了cut函数进行连续数据的等宽离散
```
pd.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False)
```
参数说明
- x：要分箱的数组或 Series。
- bins：分箱规则，可以是：
    - 整数：表示要分成多少个等宽区间。
    - 序列：表示具体的分割点。
- right：是否包含右边界，默认 True。
- labels：为分箱结果指定标签，默认返回区间对象。
- retbins：是否返回分箱的边界值。
- precision：保留小数位数。
- include_lowest：是否包含最左边界。


In [ ]:
age_cut = pd.cut(all_info['年龄'], 5)

In [ ]:
age_cut.value_counts()

### 等频法

等频分箱（Equal-frequency Binning） 是一种常见的连续型变量离散化方法。它的核心思想是：把数据按照样本数量均匀分配到若干个区间，每个区间包含的样本数大致相同。

原理:

假设有 n 个样本，需要分成 k 个区间。
- 每个区间大约包含$ \frac{n}{k} $个样本。
- 分箱的边界由数据的排序结果决定，而不是固定的数值范围。

与 **等宽分箱** 不同：
- 等宽分箱 → 区间宽度相同，但样本数可能差异很大。
- 等频分箱 → 区间样本数相同，但区间宽度可能差异很大。

特点:
- 样本分布均衡：每个箱子里样本数量接近，避免某些箱子过于稀疏。
- 区间宽度不固定：如果数据分布不均匀，某些区间可能很窄，某些区间可能很宽。
- 适合偏态分布：能保证每个分箱都有足够的数据，减少噪声影响。


In [ ]:
import numpy as np
# 自定义等频法离散化函数
def same_rate_cut(data, k):
    w = data.quantile(np.arange(0, 1 + 1.0 / k, 1.0 / k))
    data = pd.cut(data, w)
    return data

In [ ]:
age_same_rate = same_rate_cut(all_info['年龄'], 5).value_counts()

In [ ]:
age_same_rate

### 据类分析法

基于聚类的分箱（Clustering-based Binning） 是一种利用聚类算法对连续型变量进行离散化的方法。它不同于等宽分箱和等频分箱，不是简单地按照数值范围或样本数量来划分，而是通过聚类算法自动发现数据的分布结构，再将数据划分为若干类别。

原理
- 选择聚类算法：常用的有 K-means、层次聚类等。
- 设定分箱数量：比如希望分成 k 个箱子，就设定聚类的簇数为 k。
- 聚类过程：算法根据数据的相似性，把数值划分到不同的簇。
- 形成分箱：每个簇对应一个分箱，簇的边界由聚类结果决定。

特点
- 自适应性强：分箱结果由数据分布决定，更符合实际情况。
- 能捕捉复杂分布：适合数据呈现非均匀分布或多峰分布的场景。
- 比等宽/等频更智能：避免某些箱子过于稀疏或过于密集。
- 计算复杂度高：需要运行聚类算法，计算量比简单分箱大。


In [ ]:
# 自定义数据K-Means聚类离散化函数
def kmean_cut(data, k):
    from sklearn.cluster import KMeans  # 引入K-Means
    # 建立模型
    kmodel = KMeans(n_clusters=k)
    kmodel.fit(data.values.reshape((len(data), 1)))  # 训练模型
    # 输出聚类中心并排序
    c = pd.DataFrame(kmodel.cluster_centers_).sort_values(0)   
    w = c.rolling(2).mean().iloc[1:]  # 相邻两项求中点，作为边界点
    w = [0] + list(w[0]) + [data.max()]  # 把首末边界点加上
    data = pd.cut(data, w)
    return data

In [ ]:
# 用户年龄等频法离散化
age_dropna = all_info['年龄'].dropna()
age_kmeans = kmean_cut(age_dropna, 5).value_counts() 

In [ ]:
age_kmeans